In [1]:
import time, datetime
import copy
import os
import sys
import warnings

import numpy as np
from loguru import logger
import yaml
from utils import dumb_reward_plot
import gym
import assistive_gym

# sys.path.append('./envs/cartpole-envs')
# sys.path.append('./')
# import cartpole_envs
#import highway_env
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from utils import plot_reward, plot_index
from mpc.mpc_ar import MPC
from baselines.NN import NN


def load_config(config_path="config.yml"):
    if os.path.isfile(config_path):
        f = open(config_path)
        return yaml.load(f, Loader=yaml.FullLoader)
    else:
        raise Exception("Configuration file is not found in the path: "+config_path)

In [2]:
config = load_config('config/config_assistant_robot_NN.yml')
nn_config = config['NN_config']
mpc_config = config['mpc_config']
gym_config = config['gym_config']
render = gym_config['render']

# initialize the mixture model
# model = DPGPMM(dpgp_config=dpgp_config)
# model = SingleSparseGP(sparse_gp_config=sparse_gp_config)
# model = SingleGP(gp_config=gp_config)
model = NN(NN_config=nn_config)
logger.info('Using model: {}', model.name)

# initial MPC controller
mpc_controller = MPC(mpc_config=mpc_config)

# prepare task
# the task is solved, if each dynamic is solved
env = gym.make("FeedingJaco-v0")

2020-08-06 16:28:48.853 | INFO     | __main__:<module>:12 - Using model: NN
Using TensorFlow backend.
/home/baiming/anaconda3/envs/robustnp/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [3]:
"""NN pretrain"""
pretrain_episodes = 5
    # data collection
for epi in range(pretrain_episodes):
    obs = env.reset()
    done = False
    mpc_controller.reset()
    while not done:
        action = env.action_space.sample()
        obs_next, reward, done, info = env.step(action)
#             print(obs_next-obs)
        model.data_process([0, obs, action, obs_next - obs])
        obs = obs_next

model.validation_flag = True
model.fit()

7.370362254732754e-06

In [4]:
import torch
"""testing the model with MPC while training """
test_episode = 200
save_every = 10
log = []
for epi in range(test_episode):
    acc_reward = 0
    obs = env.reset()
    O, A, R, acc_reward, done = [], [], [], 0, False
    mpc_controller.reset()
    while not done:
        action = mpc_controller.act(model=model, state=obs)
        obs_next, reward, done, info = env.step(action)
#         print(obs_next)
        A.append(action)
        O.append(obs_next)
        R.append(reward)
        # append data but not training
        model.data_process([0, obs, action, obs_next - obs])
        obs = obs_next
        acc_reward += reward
#         print(obs[-1])
    print('acc_reward: ', acc_reward, 'success_food',info['task_success'], 'spilled_food',info['spilled_food'], 'distance', np.linalg.norm(obs[6:9]))
    model.fit()
    if epi % save_every == 0:
        torch.save(model.model.state_dict(), './log/ar_nn_ju_{}.pth'.format(epi))

#     if done:
#         samples = {
#             "obs": np.array(O),
#             "actions": np.array(A),
#             "rewards": np.array(R), 
#             "reward_sum": acc_reward,
#         }
#         log.append(samples)
#         if log_name is None:
#             log_name = datetime.datetime.now()
#         path = './misc/log/' + log_name.strftime("%d-%H-%M") + '.npy'
#         np.save(path, log, allow_pickle=True)
#         dumb_reward_plot(path)




acc_reward:  -154.2186206412283 success_food 0 spilled_food 0 distance 1.0251738437770324
acc_reward:  -61.43187157201522 success_food 0 spilled_food 0 distance 0.2084717424480841
acc_reward:  -125.51141801444815 success_food 0 spilled_food 0 distance 0.6416243593547212
acc_reward:  -76.04800702047527 success_food 0 spilled_food 8 distance 0.15849836175824053
acc_reward:  -52.21722246975764 success_food 0 spilled_food 0 distance 0.2066000246915539
acc_reward:  -67.21847716091862 success_food 0 spilled_food 0 distance 0.32310550988096215
acc_reward:  -108.16467581595161 success_food 0 spilled_food 0 distance 0.40426748372192983
acc_reward:  120.44007421244572 success_food 8 spilled_food 0 distance 0.08518135366400885
acc_reward:  -64.81158983547289 success_food 0 spilled_food 0 distance 0.18198915803197163
acc_reward:  -55.948098239505164 success_food 0 spilled_food 0 distance 0.16726692201518517
acc_reward:  -54.502419000587274 success_food 0 spilled_food 0 distance 0.1349684089691924


acc_reward:  -30.106173177819002 success_food 0 spilled_food 0 distance 0.07770871880694077
acc_reward:  -52.14077809039009 success_food 0 spilled_food 0 distance 0.19042628405205758
acc_reward:  -80.77229455324849 success_food 0 spilled_food 0 distance 0.3723458014107908
acc_reward:  131.6826515494688 success_food 8 spilled_food 0 distance 0.049365376692033267
acc_reward:  -47.06602712870004 success_food 0 spilled_food 0 distance 0.1579187137874312
acc_reward:  130.218207064498 success_food 8 spilled_food 0 distance 0.04487572810145913
acc_reward:  -68.42599629427582 success_food 0 spilled_food 0 distance 0.31701590144537944
acc_reward:  128.28690411242314 success_food 8 spilled_food 0 distance 0.05454098709773271
acc_reward:  111.40302745697504 success_food 8 spilled_food 0 distance 0.033433087847291536
acc_reward:  -66.50494792568165 success_food 0 spilled_food 0 distance 0.29532093051885
acc_reward:  130.79449113303863 success_food 8 spilled_food 0 distance 0.04147331259137981
acc_

acc_reward:  -40.16670408179553 success_food 0 spilled_food 0 distance 0.0602563701209409
acc_reward:  -36.68225184670142 success_food 0 spilled_food 0 distance 0.05044265691009493
acc_reward:  -47.4456540738401 success_food 0 spilled_food 0 distance 0.07516277586673975
acc_reward:  -60.67167422772276 success_food 0 spilled_food 0 distance 0.20197077178521838
acc_reward:  131.54974667129605 success_food 8 spilled_food 0 distance 0.04445050261786729
acc_reward:  -42.412749402309394 success_food 0 spilled_food 0 distance 0.07517535486295322
acc_reward:  -66.88831311776784 success_food 0 spilled_food 0 distance 0.2897474407489301
acc_reward:  -40.540026299797354 success_food 0 spilled_food 0 distance 0.08761273556118682
acc_reward:  -54.23688848163104 success_food 0 spilled_food 0 distance 0.1578133506853402
acc_reward:  119.70445025958625 success_food 8 spilled_food 0 distance 0.047219905718006064
acc_reward:  -46.362996411366204 success_food 0 spilled_food 0 distance 0.06930354133821397

In [5]:
import torch
torch.save(model.model.state_dict(), 'ar_nn.pth')